In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common
from tf_agents import specs
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.replay_buffers import py_uniform_replay_buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step

from DecisionTransformer import DecisionTransformer


2022-12-05 23:57:20.561652: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 23:57:20.619708: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-05 23:57:20.968207: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-05 23:57:20.968235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [4]:
env_name = "LunarLander-v2"
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [7]:
replay_buffer_capacity = 1000
batch_size = 32
K = 12
max_ep_len = 4096
embed_dim = 1024
n_layer = 2
n_hear = 4
activation_function = 'tanh'
dropout = True

# state_dim = (batch_size, K, 8)
state_dim = 8
# act_dim = (batch_size, K, 4)
act_dim = 4


In [8]:
dec_trans = DecisionTransformer(
    state_dim=state_dim,
    act_dim=act_dim,
    max_length=K,
    hidden_size=embed_dim,
    n_layer=n_layer,
    n_head=n_hear,
    n_inner=4*embed_dim,
    activation_function=activation_function,
    n_positions=1024,
    resid_pdrop=dropout,
    attn_pdrop=dropout,
)


In [10]:
ts = train_env.reset()
print(ts)

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[-1.3785362e-03,  1.4123918e+00, -1.3964182e-01,  6.5410286e-02,
         1.6041311e-03,  3.1630982e-02,  0.0000000e+00,  0.0000000e+00]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>})


In [ ]:
train_env.f

In [21]:
(train_env.observation_spec())

BoundedTensorSpec(shape=(8,), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32))

In [12]:
target_return = 100
ts = train_env.reset()
R, s, a, t, done = [target_return], [ts.observation], [], [1], False
while not ts.is_last():
    # sample next action
    action = dec_trans.get_action(s, a, [], R, t)
    print(action)
    ts = train_env.step(action)

    R = R + [R[-1] - ts.reward]
    s, a, t = s + ts.observation, a + action, t + len(R)
    R, s, a, t = R[-K:], s[-K:], a[-K:], t[-K:]


AttributeError: 'list' object has no attribute 'reshape'

BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(3))

In [19]:
action_tensor_spec = tensor_spec.from_spec(train_env.action_spec())
print(action_tensor_spec)

BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(3))


In [ ]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec,
    batch_size=batch_size,
    max_length=max_length)

TypeError: 'module' object is not callable

In [29]:
print(train_env.reward_spec())
print(train_env.action_spec())
print(train_env.observation_spec())
print(train_env.time_step_spec().observation)


TensorSpec(shape=(), dtype=tf.float32, name='reward')
BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(3))
BoundedTensorSpec(shape=(8,), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32))
BoundedTensorSpec(shape=(8,), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32))
